# Various topics
This notebook will cover miscellaneous topics from the image processing part of TDT4195

## Sampling and quantization

The world around us is analog or continous, while digital images are discrete. In order to produce a digital image from a continous signal we must do the two processes called sampling and quantization.

### Sampling
In image processing sampling is the process of reducing the continous spatial signal to a discrete set of pixels. Its effect can be illustrated by excessively undersampling an image and looking at the result. An undersampled image will appear pixelated. 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from ipywidgets import interact, interact_manual, FloatSlider
import ipywidgets as widgets


def sample(x):
    image = plt.imread("images/strawberry.jpg")[::2, ::2]
    x = 2**x
    plt.figure(figsize = (10,10))
    plt.imshow(image[::x, ::x])
    plt.show()
    
_ = interact(sample,x=widgets.IntSlider(min=0, max=7, step=1, value=0))

interactive(children=(IntSlider(value=0, description='x', max=7), Output()), _dom_classes=('widget-interact',)…

### Quantization
Whereas sampling is the process discretizising image in the spatial extent to a discrete set of pixels, quantization is the process of reducing the continous range of possible color intensity values to a discrete set of intensities. Often we use uint8 to represent color values. That gives us a range of 256 different color intensity levels. Other times we may require a higher integer precision for a more true representation, or we may reduce the number of possible intensity levels in order to compress the image.

In [6]:
def quantize(x):
    image = plt.imread("images/strawberry.jpg")[::2, ::2]
    x = 2**x
    plt.figure(figsize = (10,10))
    plt.imshow(image - image % x)
    plt.show()
    
_ = interact(quantize,x=widgets.IntSlider(min=0, max=7, step=1, value=0))

interactive(children=(IntSlider(value=0, description='x', max=7), Output()), _dom_classes=('widget-interact',)…

## Gamma transform
Interactive display of how the gamma-transform affects an image.

In [3]:
def gamma_transform(gamma):
    image = plt.imread("images/bw_lake.jpg")[::2,::2,0]  
    image = image/255.
    image = image**gamma
    plt.figure(figsize = (12,6))
    plt.imshow(image, cmap = "gray")
    plt.show()
    image = (image * 255).astype(np.uint8).reshape(-1)
    plt.figure(figsize = (9,6))
    _ = plt.hist(image, bins=64)
    plt.title("Image histogram")
    plt.show()

_ = interact_manual(gamma_transform,  gamma=FloatSlider(min=0.01, max=5.0, step=1e-2, value=1.0))

interactive(children=(FloatSlider(value=1.0, description='gamma', max=5.0, min=0.01, step=0.01), Button(descri…

## Log and inverse log transform
Interactive display of the log and inverse log transform

In [4]:
def log_transform(transform):
    image = plt.imread("images/high_contrast.jpg")[:,:,0]  
    image = image/255.
    if transform == "log":
        c = 1/np.log(2)
        image = c*np.log(image + 1)
    elif transform == "inverse log":
        c = 1/np.log(2)
        image = np.exp(image/c) - 1
    else:
        pass
    plt.figure(figsize=(12,6))
    plt.imshow(image, cmap = "gray")
    plt.show()
    image = (image * 255).astype(np.uint8).reshape(-1)
    plt.figure(figsize=(10,6))
    _ = plt.hist(image, bins=64)
    plt.title("Image histogram")
    plt.show()

_ = interact_manual(log_transform, transform = ["log", "identity", "inverse log"])

interactive(children=(Dropdown(description='transform', options=('log', 'identity', 'inverse log'), value='log…

## Neighborhood filtering
Interactive display of different convolution kernels. Most of the kernels can be found [here](https://en.wikipedia.org/wiki/Kernel_(image_processing)).

In [5]:
def convolve_image(image, kernel):
    h, w = image.shape[:2]
    kernel_size= kernel.shape[0]
    
    #Rotate 180 degrees in order to perform convolution and not correlation
    kernel = np.rot90(np.rot90(kernel))
    pad = (kernel_size - 1) // 2
    
    image = np.pad(image, pad_width = ((pad,pad), (pad,pad)), mode = "constant")
    
    output = np.zeros_like(image)
    for y in range(h):
        for x in range(w):
            im_slice = image[y: y+kernel_size, x: x + kernel_size]
            
            value = np.sum(im_slice*kernel, axis = (0,1))
            output[y + pad, x + pad] = value

    return output

kernel_map = {
    "identity"   : np.array([[0,0,0], [0,1,0], [0,0,0]]),
    "gaussian_3" : np.array([[1,2,1], [2,4,2], [1,2,1]])/16,
    "gaussian_5" : np.array([[1,4,6,4,1], [4,16,24,16,4],\
                             [6,24,36,24,6], [4,16,24,16,4], [1,4,6,4,1]]) /256,
    "gaussian_7" : np.array([[0,0,1,2,1,0,0], [0,3,13,22,13,3,0], [1,13,59,97,69,13,1],\
                            [2,22,97,159,97,22,2], [1,13,59,97,59,13,1], [0,3,13,22,13,3,0],\
                            [0,0,1,2,1,0,0]])/1003,
    "box_3"      : np.ones((3,3))/9,
    "box_5"      : np.ones((5,5))/25,
    "box_7"      : np.ones((7,7))/49,
    "edge_1"     : np.array([[1,0,-1], [0,0,0],[-1,0,1]]),
    "edge_2"     : np.array([[0,-1,0], [-1,4,-1], [0,-1,0]]),
    "edge_3"     : np.array([[-1,-1,-1], [-1,8,-1], [-1,-1,-1]]),
    "sharpen"    : np.array([[0,-1,0], [-1,5,-1], [0,-1,0]]),
    "sobel_x"    : np.array([[1,0,-1], [2,0,-2], [1,0,-1]]),
    "sobel_y"  : np.array([[1,2,1], [0,0,0], [-1,-2,-1]])
    }

def display_convolved_image(kernel_1, kernel_2):
    image = plt.imread("images/high_contrast.jpg")[:,:,0]
    


    output = convolve_image(image, kernel_map[kernel_1])
    output = convolve_image(output, kernel_map[kernel_2])
    plt.figure(figsize = (10,10))
    plt.imshow(output, cmap = "gray")            
    plt.show()     

_ = interact_manual(display_convolved_image, kernel_1 = kernel_map.keys(), kernel_2 = kernel_map.keys())
    

interactive(children=(Dropdown(description='kernel_1', options=('identity', 'gaussian_3', 'gaussian_5', 'gauss…